In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestClassifier

Loading Data and Separating Categorical data

In [ ]:
#DriverID = pd.read_csv('../input/traindataforproject/TrainDataCombines.csv',usecols=['DriverID']).to_numpy()
train_data = pd.read_csv('../input/traindataforproject/TrainDataCombines.csv').dropna(axis=0)
Categorical_data= train_data[['RoadCondition','PrecipitationType','PrecipitationIntensity','LightingCondition']].to_numpy()
target_data=train_data['DrivingStyle'].to_numpy()
DriverID = train_data['DriverID'].to_numpy().reshape([len(train_data['DriverID']),1])

train_data = np.nan_to_num(train_data.drop(columns=['DrivingStyle','IDOfPrecedingVehicle','DateTimeOfTrip','DriverID','RoadCondition','PrecipitationType','PrecipitationIntensity','LightingCondition']).to_numpy())
#Categorical_data=pd.read_csv('../input/traindataforproject/TrainDataCombines.csv',usecols=['RoadCondition','PrecipitationType','PrecipitationIntensity','LightingCondition'])


One Hot Encoding for the categorical data

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')

#Categorical_data=pd.read_csv('../input/traindataforproject/TrainDataCombines.csv',usecols=['RoadCondition','PrecipitationType','PrecipitationIntensity','LightingCondition'])

enc.fit(Categorical_data)
#Categorical_data_new = OneHotEncoder(handle_unknown='ignore').fit_transform(Categorical_data).toarray()
enc.categories_
#print(Categorical_data_new)
Categorical_data_new = enc.transform(Categorical_data).toarray()
#print(Categorical_data_new)

train_data=np.hstack((train_data,Categorical_data_new))
#target_data=pd.read_csv('../input/traindataforproject/TrainDataCombines.csv',usecols=['DrivingStyle']).to_numpy()



Scaling of the data

In [ ]:
scaler = StandardScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
X_train, X_test, y_train, y_test = train_test_split(train_data, target_data, test_size=0.33, random_state=42)

In [ ]:
THIS IS FOR TRAINING WITHOUT SEPARATING OF DATA AS PER DRIVERS

MLP =MLPClassifier(hidden_layer_sizes = (100,100),activation = 'relu', solver = 'sgd',alpha = 0, batch_size = 128,learning_rate = 'constant',learning_rate_init = 1e-3,max_iter = 250,shuffle = False,momentum = 0, nesterovs_momentum = False,early_stopping = False,validation_fraction = 0, verbose  = True)

MLP.fit(X_train, y_train.ravel())
#predictions_train = MLP.predict(X_train)
predictions_test = MLP.predict(X_test)
print(confusion_matrix(y_test,predictions_test))
print(classification_report(y_test,predictions_test))


Separating data as per DriverID

In [ ]:
driver_data = pd.read_csv('../input/traindataforprojectoriginal/Train.csv',usecols=['ID','DrivingStyle']).to_numpy()
print(driver_data)
Xd_train, Xd_rem = train_test_split(driver_data, test_size=0.40,shuffle = True,  random_state=42)
#print(Xd_train)
#print(Xd_rem)
Xd_dev, Xd_test = train_test_split(Xd_rem, test_size=0.50, shuffle = True, random_state=42)
print(Xd_train)
print(Xd_dev)
print(Xd_rem)

In [ ]:
Xtot_train = []
Ytot_train= []
train_with_driver = np.hstack((DriverID, train_data))
for i in range(len(train_with_driver)):
    if train_with_driver[i,0] in Xd_train[:,0]:
        Xtot_train.append(train_with_driver[i,:])
        Ytot_train.append(target_data[i])
        
Xtot_train = np.vstack(Xtot_train)
Ytot_train = np.vstack(Ytot_train)
print(np.shape(Xtot_train), np.shape(Ytot_train))

Xtot_dev = []
Ytot_dev = []
for i in range(len(train_with_driver)):
    if train_with_driver[i,0] in Xd_dev[:,0]:
        Xtot_dev.append(train_with_driver[i,:])
        Ytot_dev.append(target_data[i])


Xtot_dev = np.vstack(Xtot_dev)
Ytot_dev = np.vstack(Ytot_dev)
print(np.shape(Xtot_dev),np.shape(Ytot_dev))

Xtot_test = []
Ytot_test = []
for i in range(len(train_with_driver)):
    if train_with_driver[i,0] in Xd_test[:,0]:
        Xtot_test.append(train_with_driver[i,:])
        Ytot_test.append(target_data[i])


Xtot_test = np.vstack(Xtot_test)
Ytot_test = np.vstack(Ytot_test)
print(np.shape(Xtot_test),np.shape(Ytot_test))
        
    

In [ ]:
#MLP =MLPClassifier(hidden_layer_sizes = (100,100,100),activation = 'relu', solver = 'sgd',alpha = 0, batch_size = 128,learning_rate = 'constant',learning_rate_init = 1e-4,max_iter = 250,shuffle = False,momentum = 0, nesterovs_momentum = False,early_stopping = False,validation_fraction = 0, verbose  = True)
MLP =MLPClassifier(hidden_layer_sizes = (100,100),activation = 'relu', solver = 'adam',alpha = 0.1, batch_size = 128,learning_rate = 'adaptive',learning_rate_init = 1e-4,max_iter = 500,shuffle = False,momentum = 0, nesterovs_momentum = False,early_stopping = False,validation_fraction = 0, verbose  = True)
#MLP =MLPRegressor(hidden_layer_sizes = (64,64,64),activation = 'relu', solver = 'sgd',alpha = 0, batch_size = 128,learning_rate = 'constant',learning_rate_init = 1e-6,max_iter = 500,shuffle = False,momentum = 0, nesterovs_momentum = False,early_stopping = False,validation_fraction = 0, verbose  = True)
MLP.fit(Xtot_train[:,1:], Ytot_train.ravel())
#predictions_train = MLP.predict(X_train)
predictions_dev = MLP.predict(Xtot_dev[:,1:])


print(confusion_matrix(Ytot_dev,predictions_dev))
print(classification_report(Ytot_dev,predictions_dev))

predictions_test = MLP.predict(Xtot_test[:,1:])


print(confusion_matrix(Ytot_test,predictions_test))
print(classification_report(Ytot_test,predictions_test))




In [ ]:
# Hidden Layer Sizes
hidden_layer_sizes = [(100),(100,100),(100,100,100),(50),(50,50),(50,50,50)]
#Solver
solver = ['sgd', 'adam']
# Maximum number of levels in tree
# max_depth = [2,4]
batch_size = [128,256]
#alpha = [0.1,0.01,0.001]
learning_rate = ['adaptive']

learning_rate_init = [0.01,0.001,0.0001]
# Minimum number of samples required to split a node
# min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2]

# Method of selecting samples for training each tree
#bootstrap = [True, False]

param_grid = {'hidden_layer_sizes':hidden_layer_sizes,
               'solver': solver,
                'batch_size': batch_size,
                'learning_rate': learning_rate,
     #            'alpha': alpha,
               'learning_rate_init': learning_rate_init}

MLPC_Model =MLPClassifier()

MLPC_Grid = GridSearchCV(estimator = MLPC_Model, param_grid = param_grid, cv = 5, verbose=10, n_jobs = 4)

MLPC_Grid.fit(Xtot_train, Ytot_train)

MLPC_Grid.best_params_

In [ ]:
print (f'Train Accuracy - : {MLPC_Grid.score(Xtot_train,Ytot_train):.3f}')
print (f'Test Accuracy - : {MLPC_Grid.score(Xtot_dev,Ytot_dev):.3f}')
print (f'Test Accuracy - : {MLPC_Grid.score(Xtot_test,Ytot_test):.3f}')